# Shapefile 字段名称批量重命名工具

本工具用于**批量修改 Shapefile 属性表的字段名称**，特别适合将中文字段名转换为英文字段名，以满足 GIS 软件和数据库的命名规范。

## 主要功能

1. 中文字段名批量转换为英文（通过映射表）
2. 自动处理 Shapefile 字段名长度限制（最多 10 个字符）
3. 自动清理非法字符（替换为下划线）
4. 保留几何字段和未映射的字段
5. 输出字段映射关系日志

## 应用场景

- 数据标准化处理
- 数据库导入前的字段名规范化
- 跨平台数据交换（避免中文乱码）
- GIS 软件兼容性处理

> **依赖库**  
> - GeoPandas
> - re（正则表达式）
> 
> 安装命令：  
> ```bash
> pip install geopandas
> ```

In [2]:
# pip install geopandas shapely fiona pyproj
import re
from pathlib import Path
import geopandas as gpd

# ========= 1. 配置 =========
IN_PATH  = r"E:\项目管理\深泽\深泽\数据\export_result_DE6957DBB80345118D432806BB268BAC\y_gis0573_管段.shp"       # 输入路径（你的原始 shp）
OUT_PATH = r"E:\项目管理\深泽\深泽\数据\gx\guanduan.shp"   # 输出路径（新的 shp）

# 中文 -> 英文字段映射表
CN2EN = {
    "管径": "diameter",
    "埋深": "burial_depth",
    # 可以继续补充，比如：
    # "系统类型": "systemtype",
    # "起点编号": "us_node",
    # "终点编号": "ds_node",
    "管材": "material",
    "压力级别":"press_lev"
}

# ========= 2. 读取 shp =========
gdf = gpd.read_file(IN_PATH)   # 如遇乱码：encoding="gbk"

# ========= 3. 重命名字段 =========
rename_map = {}
for col in gdf.columns:
    if col == gdf.geometry.name:
        continue
    new_name = CN2EN.get(col, col)  # 没在映射表里的字段保持原名
    # shapefile 字段名最多 10 字符
    new_name = re.sub(r"[^\w]", "_", new_name)[:10]
    rename_map[col] = new_name

gdf_renamed = gdf.rename(columns=rename_map)

# ========= 4. 保存新的 shp =========
Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)
gdf_renamed.to_file(OUT_PATH, encoding="utf-8")

print(f"[DONE] 已生成新的 shp：{OUT_PATH}")
print("字段映射如下：")
for k, v in rename_map.items():
    if k != v:
        print(f"  {k} → {v}")


C:\Users\lishuhao\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: One or several characters couldn't be converted correctly from GB2312 to UTF-8.  This warning will not be emitted anymore
  return ogr_read(


[DONE] 已生成新的 shp：E:\项目管理\深泽\深泽\数据\gx\guanduan.shp
字段映射如下：
  压力级别 → press_lev
  管径 → diameter
  管材 → material
  埋深 → burial_dep


## 配置说明

在 `CN2EN` 字典中定义中文到英文的字段名映射关系。